In [3]:
import polars as pl
from pathlib import Path

In [5]:
DATA_DIR   = Path("../data/raw/")         
TRAIN_PQ   = DATA_DIR / "train.parquet"
VAL_PQ     = DATA_DIR / "validation.parquet"
TEST_PQ    = DATA_DIR / "test.parquet"

In [6]:
train_pl = pl.read_parquet(TRAIN_PQ)
val_pl   = pl.read_parquet(VAL_PQ)
test_pl  = pl.read_parquet(TEST_PQ)


In [7]:
for name, df in [("train", train_pl), ("val", val_pl), ("test", test_pl)]:
    print(f"{name:5s} → rows={df.height:,}  cols={df.width}")
    print(df.schema, "\n")


train → rows=163,168  cols=5
Schema([('id', String), ('title', String), ('abstract', String), ('categories', List(String)), ('creation_date', Datetime(time_unit='ns', time_zone='UTC'))]) 

val   → rows=20,396  cols=5
Schema([('id', String), ('title', String), ('abstract', String), ('categories', List(String)), ('creation_date', Datetime(time_unit='ns', time_zone='UTC'))]) 

test  → rows=20,397  cols=5
Schema([('id', String), ('title', String), ('abstract', String), ('categories', List(String)), ('creation_date', Datetime(time_unit='ns', time_zone='UTC'))]) 



In [10]:
def null_report(df: pl.DataFrame):
    return (df
            .null_count().with_columns(pl.col("*").cast(pl.Float32)/df.height))


null_report(train_pl)
null_report(val_pl)
null_report(test_pl)

id,title,abstract,categories,creation_date
f32,f32,f32,f32,f32
0.0,0.0,0.0,0.0,0.0


In [13]:
train_pl.group_by("categories").len().sort("len", descending=True)

categories,len
list[str],u32
"[""Physics Archive->astro-ph""]",7649
"[""Physics Archive->hep->hep-ph""]",7059
"[""Physics Archive->quant-ph""]",5865
"[""Physics Archive->hep->hep-th""]",5109
"[""Computer Science Archive->cs.CV""]",4595
…,…
"[""Mathematics Archive->math.FA"", ""Mathematics Archive->math.PR""]",81
"[""Physics Archive->physics->physics.data-an""]",81
"[""Physics Archive->hep->hep-ex"", ""Physics Archive->hep->hep-lat"", ""Physics Archive->hep->hep-ph""]",81


In [18]:
(train_pl
 .group_by("categories")
 .agg(pl.count().alias("count"))
 .sort("count", descending=True)
 .head(20)
 .select(["count", "categories"])
 .write_parquet("../data/samples/snapshot_sample.parquet"))

/var/folders/pg/sp9kdgzs23z2bvfm7_sn7p640000gn/T/ipykernel_88151/3131108015.py:3: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  .agg(pl.count().alias("count"))
